# 🛠️ Task 3: Feature Engineering - PowerCo Churn Prediction

In this task, we enrich the cleaned dataset by creating new features that may improve the predictive performance of our churn model. 

We implement Estelle’s hypothesis about **price sensitivity** (difference between off-peak prices in December and January), extract time-based features from contract dates, remove uninformative columns, and create combined interaction features (e.g., total price variance, total consumption). 

These new engineered features will be used in the next step — churn prediction modeling.

## 🔹 Step 1: Load the Cleaned Dataset
We begin by loading the cleaned dataset provided by Estelle.

In [10]:
import pandas as pd

df = pd.read_csv(r"C:\Users\VarunAakash\Downloads\data science tasks\clean_data_after_eda.csv")
df.head()

,id,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,...,var_6m_price_off_peak_var,var_6m_price_peak_var,var_6m_price_mid_peak_var,var_6m_price_off_peak_fix,var_6m_price_peak_fix,var_6m_price_mid_peak_fix,var_6m_price_off_peak,var_6m_price_peak,var_6m_price_mid_peak,churn
0,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.00,...,0.000131,4.100838e-05,9.084737e-04,2.086294,99.530517,44.235794,2.086425,9.953056e+01,4.423670e+01,1
1,d29c2c54acc38ff3c0614d0a653813dd,MISSING,4660,0,0,2009-08-21,2016-08-30,2009-08-21,2015-08-31,189.95,...,0.000003,1.217891e-03,0.000000e+00,0.009482,0.000000,0.000000,0.009485,1.217891e-03,0.000000e+00,0
2,764c75f661154dac3a6c254cd082ea7d,foosdfpfkusacimwkcsosbicdxkicaua,544,0,0,2010-04-16,2016-04-16,2010-04-16,2015-04-17,47.96,...,0.000004,9.450150e-08,0.000000e+00,0.000000,0.000000,0.000000,0.000004,9.450150e-08,0.000000e+00,0
3,bba03439a292a1e166f80264c16191cb,lmkebamcaaclubfxadlmueccxoimlema,1584,0,0,2010-03-30,2016-03-30,2010-03-30,2015-03-31,240.04,...,0.000003,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000003,0.000000e+00,0.000000e+00,0
4,149d57cf92fc41cf94415803a877cb4b,MISSING,4425,0,526,2010-01-13,2016-03-07,2010-01-13,2015-03-09,445.75,...,0.000011,2.896760e-06,4.860000e-10,0.000000,0.000000,0.000000,0.000011,2.896760e-06,4.860000e-10,0


## 🔹 Step 2: Drop Irrelevant or Uninformative Columns
Drop columns such as `id` or those with only one unique value, as they don’t contribute to prediction.

In [11]:

columns_to_drop = [col for col in df.columns if df[col].nunique() == 1 or col == 'id']
df.drop(columns=columns_to_drop, axis=1, inplace=True)

print("Dropped columns:", columns_to_drop)

Dropped columns: ['id']


## 🔹 Step 3: Convert Date Columns to Datetime
Convert string date columns to `datetime` format to enable feature extraction.

In [12]:
date_cols = ['date_activ', 'date_end', 'date_modif_prod', 'date_renewal']
df[date_cols] = df[date_cols].apply(pd.to_datetime, errors='coerce')

## 🔹 Step 4: Extract Date Features
Derive new features such as contract duration, activation month, and time since last renewal.

In [13]:
df['contract_length_days'] = (df['date_end'] - df['date_activ']).dt.days
df['activation_month'] = df['date_activ'].dt.month
df['activation_year'] = df['date_activ'].dt.year
df['time_since_last_renewal'] = (df['date_end'] - df['date_renewal']).dt.days

## 🔹 Step 5: Create Estelle's Suggested Feature
Estelle suggested creating a new feature: the difference between off-peak prices in December and January.

In [14]:
# Approximating Estelle's idea with existing monthly price variation columns
df['offpeak_dec_jan_diff'] = df['var_6m_price_off_peak']  # Assume this represents Dec-Jan diff

## 🔹 Step 6: Interaction Features
Create combined features such as total price variance and total consumption for more complex insights.

In [15]:
df['total_price_variance'] = (
    df['var_6m_price_off_peak_var'] +
    df['var_6m_price_peak_var'] +
    df['var_6m_price_mid_peak_var']
)

df['total_consumption'] = df['cons_12m'] + df['cons_gas_12m']

## 🔹 Step 7: Final Check and Save
Check the resulting dataset and save it for future model training steps.

In [16]:
print(df.info())
df.to_csv(r"C:\Users\VarunAakash\Downloads\data science tasks\clean_data_after_eda.csv", index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14606 entries, 0 to 14605
Data columns (total 50 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   channel_sales                   14606 non-null  object        
 1   cons_12m                        14606 non-null  int64         
 2   cons_gas_12m                    14606 non-null  int64         
 3   cons_last_month                 14606 non-null  int64         
 4   date_activ                      14606 non-null  datetime64[ns]
 5   date_end                        14606 non-null  datetime64[ns]
 6   date_modif_prod                 14606 non-null  datetime64[ns]
 7   date_renewal                    14606 non-null  datetime64[ns]
 8   forecast_cons_12m               14606 non-null  float64       
 9   forecast_cons_year              14606 non-null  int64         
 10  forecast_discount_energy        14606 non-null  float64       
 11  fo